# Versión con Ajuste Fino (Fine-Tuning) para el modelo multi-salida

Esta versión del código implementa el ajuste fino descongelando las últimas capas del modelo base EfficientNetB0 para mejorar la adaptación a tu conjunto de datos.

In [2]:
# Importar librerías necesarias
import os
import json
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

In [3]:
# =============================================
# CONFIGURACIÓN
# =============================================
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS_INITIAL = 30  # Entrenamiento inicial con base congelada
EPOCHS_FINE_TUNE = 30  # Entrenamiento con ajuste fino
LEARNING_RATE_INITIAL = 1e-4
LEARNING_RATE_FINE_TUNE = 1e-5
BASE_MODEL = 'EfficientNetB0'

In [5]:
# =============================================
# DICCIONARIOS DE MAPEO
# =============================================
label_to_cls_piezas = {
    1: "Antiniebla delantero derecho",
    2: "Antiniebla delantero izquierdo",
    3: "Capó",
    4: "Cerradura capo",
    5: "Cerradura maletero",
    6: "Cerradura puerta",
    7: "Espejo lateral derecho",
    8: "Espejo lateral izquierdo",
    9: "Faros derecho",
    10: "Faros izquierdo",
    11: "Guardabarros delantero derecho",
    12: "Guardabarros delantero izquierdo",
    13: "Guardabarros trasero derecho",
    14: "Guardabarros trasero izquierdo",
    15: "Luz indicadora delantera derecha",
    16: "Luz indicadora delantera izquierda",
    17: "Luz indicadora trasera derecha",
    18: "Luz indicadora trasera izquierda",
    19: "Luz trasera derecho",
    20: "Luz trasera izquierdo",
    21: "Maletero",
    22: "Manija derecha",
    23: "Manija izquierda",
    24: "Marco de la ventana",
    25: "Marco de las puertas",
    26: "Moldura capó",
    27: "Moldura puerta delantera derecha",
    28: "Moldura puerta delantera izquierda",
    29: "Moldura puerta trasera derecha",
    30: "Moldura puerta trasera izquierda",
    31: "Parabrisas delantero",
    32: "Parabrisas trasero",
    33: "Parachoques delantero",
    34: "Parachoques trasero",
    35: "Puerta delantera derecha",
    36: "Puerta delantera izquierda",
    37: "Puerta trasera derecha",
    38: "Puerta trasera izquierda",
    39: "Rejilla, parrilla",
    40: "Rueda",
    41: "Tapa de combustible",
    42: "Tapa de rueda",
    43: "Techo",
    44: "Techo corredizo",
    45: "Ventana delantera derecha",
    46: "Ventana delantera izquierda",
    47: "Ventana trasera derecha",
    48: "Ventana trasera izquierda",
    49: "Ventanilla delantera derecha",
    50: "Ventanilla delantera izquierda",
    51: "Ventanilla trasera derecha",
    52: "Ventanilla trasera izquierda"
}

label_to_cls_danos = {
    1: "Abolladura",
    2: "Deformación",
    3: "Desprendimiento",
    4: "Fractura",
    5: "Rayón",
    6: "Rotura"
}

label_to_cls_sugerencias = {
    1: "Reparar",
    2: "Reemplazar"
}

In [7]:
# =============================================
# CARGA DE DATOS
# =============================================
def load_and_prepare_data(split_dir):
    """Carga imágenes y etiquetas para un conjunto (train/val/test)"""
    df = pd.read_csv(f'{split_dir}/{split_dir.split("/")[-1]}_set.csv', sep='|')
    
    # Convertir strings de listas a listas reales
    for col in ['dannos', 'partes', 'sugerencias']:
        df[col] = df[col].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
    
    return df

# Cargar todos los conjuntos
train_df = load_and_prepare_data('data/fotos_siniestros/train')
val_df = load_and_prepare_data('data/fotos_siniestros/val')
test_df = load_and_prepare_data('data/fotos_siniestros/test')

In [8]:
# =============================================
# PREPARACIÓN DE ETIQUETAS
# =============================================
mlb_partes = MultiLabelBinarizer()
mlb_dannos = MultiLabelBinarizer()
mlb_sugerencias = MultiLabelBinarizer()

y_train_partes = mlb_partes.fit_transform(train_df['partes'])
y_train_dannos = mlb_dannos.fit_transform(train_df['dannos'])
y_train_sugerencias = mlb_sugerencias.fit_transform(train_df['sugerencias'])

y_val_partes = mlb_partes.transform(val_df['partes'])
y_val_dannos = mlb_dannos.transform(val_df['dannos'])
y_val_sugerencias = mlb_sugerencias.transform(val_df['sugerencias'])

In [9]:
# =============================================
# GENERADOR DE DATOS (igual que en tu código actual)
# =============================================
class MultiLabelDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, img_dir, mlb_partes, mlb_dannos, mlb_sugerencias, 
                 batch_size=32, img_size=(224, 224), shuffle=True, augmentation=None):
        self.df = df
        self.img_dir = img_dir
        self.mlb_partes = mlb_partes
        self.mlb_dannos = mlb_dannos
        self.mlb_sugerencias = mlb_sugerencias
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch_df = self.df.iloc[batch_indices]
        
        X = np.empty((len(batch_df), *self.img_size, 3))
        y_partes = []
        y_dannos = []
        y_sugerencias = []
        
        for i, (_, row) in enumerate(batch_df.iterrows()):
            img_path = os.path.join(self.img_dir, row['Imagen'])
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=self.img_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            
            if self.augmentation:
                img_array = self.augmentation.random_transform(img_array)
                
            img_array = applications.efficientnet.preprocess_input(img_array)
            X[i] = img_array
            
            y_partes.append(row['partes'])
            y_dannos.append(row['dannos'])
            y_sugerencias.append(row['sugerencias'])
        
        y_partes = np.array(self.mlb_partes.transform(y_partes), dtype='float32')
        y_dannos = np.array(self.mlb_dannos.transform(y_dannos), dtype='float32')
        y_sugerencias = np.array(self.mlb_sugerencias.transform(y_sugerencias), dtype='float32')
        
        return X, {'partes': y_partes, 'dannos': y_dannos, 'sugerencias': y_sugerencias}
    
    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

IMG_DIR = '../data/fotos_siniestros/'

train_generator = MultiLabelDataGenerator(
    train_df, 
    IMG_DIR, 
    mlb_partes, 
    mlb_dannos, 
    mlb_sugerencias,
    batch_size=BATCH_SIZE, 
    img_size=IMG_SIZE,
    augmentation=train_datagen
)

val_generator = MultiLabelDataGenerator(
    val_df, IMG_DIR, mlb_partes, mlb_dannos, mlb_sugerencias, 
    batch_size=BATCH_SIZE, img_size=IMG_SIZE, shuffle=False
)

test_generator = MultiLabelDataGenerator(
    test_df, IMG_DIR, mlb_partes, mlb_dannos, mlb_sugerencias, 
    batch_size=BATCH_SIZE, img_size=IMG_SIZE, shuffle=False
)

In [ ]:
# =============================================
# CONSTRUCCIÓN DEL MODELO CON AJUSTE FINO
# =============================================
# Last Execution 12:30:31 PM
# Execution Time 11m 58.7s
# Overhead Time 104ms
# Render Times
# VS Code Builtin Notebook Output Renderer 69ms

def build_multi_output_model_fine_tuning(input_shape, num_partes, num_dannos, num_sugerencias, base_model_name='EfficientNetB0'):
    if base_model_name == 'EfficientNetB0':
        base_model = applications.EfficientNetB0(
            include_top=False, 
            weights='imagenet', 
            input_shape=input_shape
        )
    elif base_model_name == 'ResNet50':
        base_model = applications.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=input_shape
        )
    else:
        raise ValueError(f"Modelo base no soportado: {base_model_name}")

    # Congelar todas las capas inicialmente
    base_model.trainable = False

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    output_partes = layers.Dense(num_partes, activation='sigmoid', name='partes')(x)
    output_dannos = layers.Dense(num_dannos, activation='sigmoid', name='dannos')(x)
    output_sugerencias = layers.Dense(num_sugerencias, activation='sigmoid', name='sugerencias')(x)

    model = models.Model(
        inputs=inputs,
        outputs=[output_partes, output_dannos, output_sugerencias]
    )

    return model

# Construir modelo
model = build_multi_output_model_fine_tuning(
    input_shape=(*IMG_SIZE, 3),
    num_partes=len(mlb_partes.classes_),
    num_dannos=len(mlb_dannos.classes_),
    num_sugerencias=len(mlb_sugerencias.classes_),
    base_model_name=BASE_MODEL
)

# Compilar para entrenamiento inicial
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_INITIAL),
    loss={
        'partes': 'binary_crossentropy',
        'dannos': 'binary_crossentropy',
        'sugerencias': 'binary_crossentropy'
    },
    metrics={
        'partes': 'accuracy',
        'dannos': 'accuracy',
        'sugerencias': 'accuracy'
    }
)

# Callbacks
checkpoint = ModelCheckpoint(
    'best_model_fine_tuning.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Entrenamiento inicial con base congelada
history_initial = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_INITIAL,
    callbacks=[checkpoint, early_stopping],
    class_weight=None,
    verbose=1
)

# Descongelar últimas capas para ajuste fino
base_model = model.layers[1]  # Asumiendo que la segunda capa es el modelo base
base_model.trainable = True

# Congelar todas las capas excepto las últimas 20
for layer in base_model.layers[:-20]:
    layer.trainable = False

# Recompilar con tasa de aprendizaje menor
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE_FINE_TUNE),
    loss={
        'partes': 'binary_crossentropy',
        'dannos': 'binary_crossentropy',
        'sugerencias': 'binary_crossentropy'
    },
    metrics={
        'partes': 'accuracy',
        'dannos': 'accuracy',
        'sugerencias': 'accuracy'
    }
)

# Entrenamiento con ajuste fino
history_fine_tune = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS_FINE_TUNE,
    callbacks=[checkpoint, early_stopping],
    class_weight=None,
    verbose=1
)

# Guardar modelo final
model.save('final_model_fine_tuned.keras')

2025-05-07 12:18:32.247504: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/data/Python/DetectarDannosPartesSugerenciasUsandoMultiplesEtiquetasTF_APIs/.venv-3-11/lib64/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - dannos_accuracy: 0.1566 - dannos_loss: 0.6620 - loss: 2.0374 - partes_accuracy: 0.0590 - partes_loss: 0.6996 - sugerencias_accuracy: 0.5835 - sugerencias_loss: 0.6744
Epoch 1: val_loss improved from inf to 1.54802, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 23s 705ms/step - dannos_accuracy: 0.1582 - dannos_loss: 0.6594 - loss: 2.0311 - partes_accuracy: 0.0587 - partes_loss: 0.6975 - sugerencias_accuracy: 0.5872 - sugerencias_loss: 0.6724 - val_dannos_accuracy: 0.2632 - val_dannos_loss: 0.4966 - val_loss: 1.5480 - val_partes_accuracy: 0.0526 - val_partes_loss: 0.5299 - val_sugerencias_accuracy: 0.8304 - val_sugerencias_loss: 0.5333
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 551ms/step - dannos_accuracy: 0.2880 - dannos_loss: 0.4932 - loss: 1.5189 - partes_accuracy: 0.0663 - partes_loss: 0.4814 - sugerencias_accuracy: 0.8104 - sugerencias_loss: 0.5545
Epoch 2: val_loss improved from 1.54802 to 1.31243, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 17s 665ms/step - dannos_accuracy: 0.2891 - dannos_loss: 0.4926 - loss: 1.5163 - partes_accuracy: 0.0669 - partes_loss: 0.4800 - sugerencias_accuracy: 0.8105 - sugerencias_loss: 0.5542 - val_dannos_accuracy: 0.3099 - val_dannos_loss: 0.4502 - val_loss: 1.3124 - val_partes_accuracy: 0.1228 - val_partes_loss: 0.3968 - val_sugerencias_accuracy: 0.8830 - val_sugerencias_loss: 0.4753
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 575ms/step - dannos_accuracy: 0.3517 - dannos_loss: 0.4550 - loss: 1.3029 - partes_accuracy: 0.0827 - partes_loss: 0.3582 - sugerencias_accuracy: 0.8534 - sugerencias_loss: 0.5091
Epoch 3: val_loss improved from 1.31243 to 1.18497, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 18s 693ms/step - dannos_accuracy: 0.3521 - dannos_loss: 0.4549 - loss: 1.3019 - partes_accuracy: 0.0832 - partes_loss: 0.3575 - sugerencias_accuracy: 0.8533 - sugerencias_loss: 0.5089 - val_dannos_accuracy: 0.3743 - val_dannos_loss: 0.4316 - val_loss: 1.1850 - val_partes_accuracy: 0.1871 - val_partes_loss: 0.3277 - val_sugerencias_accuracy: 0.8772 - val_sugerencias_loss: 0.4328
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 573ms/step - dannos_accuracy: 0.3819 - dannos_loss: 0.4495 - loss: 1.2002 - partes_accuracy: 0.1140 - partes_loss: 0.3029 - sugerencias_accuracy: 0.8393 - sugerencias_loss: 0.4497
Epoch 4: val_loss improved from 1.18497 to 1.10510, saving model to best_model_fine_tuning.h5


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 631ms/step - dannos_accuracy: 0.6178 - dannos_loss: 0.3229 - loss: 0.8179 - partes_accuracy: 0.3160 - partes_loss: 0.2155 - sugerencias_accuracy: 0.8780 - sugerencias_loss: 0.2660 - val_dannos_accuracy: 0.6433 - val_dannos_loss: 0.3273 - val_loss: 0.8457 - val_partes_accuracy: 0.3860 - val_partes_loss: 0.2016 - val_sugerencias_accuracy: 0.8947 - val_sugerencias_loss: 0.3303
Epoch 26/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 525ms/step - dannos_accuracy: 0.6584 - dannos_loss: 0.3117 - loss: 0.8014 - partes_accuracy: 0.2955 - partes_loss: 0.2172 - sugerencias_accuracy: 0.8804 - sugerencias_loss: 0.2866
Epoch 26: val_loss improved from 0.84569 to 0.84537, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 627ms/step - dannos_accuracy: 0.6574 - dannos_loss: 0.3120 - loss: 0.8015 - partes_accuracy: 0.2958 - partes_loss: 0.2172 - sugerencias_accuracy: 0.8803 - sugerencias_loss: 0.2866 - val_dannos_accuracy: 0.6374 - val_dannos_loss: 0.3242 - val_loss: 0.8454 - val_partes_accuracy: 0.3977 - val_partes_loss: 0.2007 - val_sugerencias_accuracy: 0.8889 - val_sugerencias_loss: 0.3307
Epoch 27/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - dannos_accuracy: 0.6455 - dannos_loss: 0.3125 - loss: 0.8046 - partes_accuracy: 0.3034 - partes_loss: 0.2108 - sugerencias_accuracy: 0.8743 - sugerencias_loss: 0.2736
Epoch 27: val_loss improved from 0.84537 to 0.84284, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 619ms/step - dannos_accuracy: 0.6453 - dannos_loss: 0.3123 - loss: 0.8041 - partes_accuracy: 0.3039 - partes_loss: 0.2106 - sugerencias_accuracy: 0.8743 - sugerencias_loss: 0.2733 - val_dannos_accuracy: 0.6257 - val_dannos_loss: 0.3226 - val_loss: 0.8428 - val_partes_accuracy: 0.3977 - val_partes_loss: 0.1992 - val_sugerencias_accuracy: 0.8947 - val_sugerencias_loss: 0.3308
Epoch 28/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 547ms/step - dannos_accuracy: 0.6125 - dannos_loss: 0.3470 - loss: 0.8017 - partes_accuracy: 0.3080 - partes_loss: 0.2217 - sugerencias_accuracy: 0.8721 - sugerencias_loss: 0.2952
Epoch 28: val_loss did not improve from 0.84284
26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 645ms/step - dannos_accuracy: 0.6125 - dannos_loss: 0.3460 - loss: 0.8013 - partes_accuracy: 0.3084 - partes_loss: 0.2212 - sugerencias_accuracy: 0.8721 - sugerencias_loss: 0.2948 - val_dannos_accuracy: 0.6433 - val_dannos_loss: 0.3219 - val_loss: 0.8455 - val_partes_accuracy: 0.3977 - val_

26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 627ms/step - dannos_accuracy: 0.6283 - dannos_loss: 0.3228 - loss: 0.8121 - partes_accuracy: 0.3320 - partes_loss: 0.1969 - sugerencias_accuracy: 0.8768 - sugerencias_loss: 0.2894 - val_dannos_accuracy: 0.6374 - val_dannos_loss: 0.3222 - val_loss: 0.8412 - val_partes_accuracy: 0.3801 - val_partes_loss: 0.1972 - val_sugerencias_accuracy: 0.8947 - val_sugerencias_loss: 0.3345
Epoch 30/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 518ms/step - dannos_accuracy: 0.6099 - dannos_loss: 0.2918 - loss: 0.7658 - partes_accuracy: 0.3351 - partes_loss: 0.1994 - sugerencias_accuracy: 0.8768 - sugerencias_loss: 0.2746
Epoch 30: val_loss improved from 0.84118 to 0.84010, saving model to best_model_fine_tuning.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 620ms/step - dannos_accuracy: 0.6112 - dannos_loss: 0.2919 - loss: 0.7656 - partes_accuracy: 0.3354 - partes_loss: 0.1995 - sugerencias_accuracy: 0.8772 - sugerencias_loss: 0.2743 - val_dannos_accuracy: 0.6433 - val_dannos_loss: 0.3209 - val_loss: 0.8401 - val_partes_accuracy: 0.4094 - val_partes_loss: 0.1958 - val_sugerencias_accuracy: 0.9006 - val_sugerencias_loss: 0.3338
Restoring model weights from the end of the best epoch: 30.
Epoch 1/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - dannos_accuracy: 0.5426 - dannos_loss: 0.4408 - loss: 1.1822 - partes_accuracy: 0.3106 - partes_loss: 0.3205 - sugerencias_accuracy: 0.7414 - sugerencias_loss: 0.4352
Epoch 1: val_loss did not improve from 0.84010
26/26 ━━━━━━━━━━━━━━━━━━━━ 24s 717ms/step - dannos_accuracy: 0.5432 - dannos_loss: 0.4404 - loss: 1.1811 - partes_accuracy: 0.3105 - partes_loss: 0.3203 - sugerencias_accuracy: 0.7427 - sugerencias_loss: 0.4345 - val_dannos_accuracy: 0.6374 - val_dannos_loss: 0.32

In [ ]:
# Función para visualizar el historial de entrenamiento
def plot_training_history(history1, history2):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history1.history['loss'], label='Train Loss Initial')
    plt.plot(history1.history['val_loss'], label='Val Loss Initial')
    plt.plot(history2.history['loss'], label='Train Loss Fine Tune')
    plt.plot(history2.history['val_loss'], label='Val Loss Fine Tune')
    plt.title('Loss Evolution')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history1.history['partes_accuracy'], label='Train Parts Accuracy Initial')
    plt.plot(history1.history['val_partes_accuracy'], label='Val Parts Accuracy Initial')
    plt.plot(history2.history['partes_accuracy'], label='Train Parts Accuracy Fine Tune')
    plt.plot(history2.history['val_partes_accuracy'], label='Val Parts Accuracy Fine Tune')
    plt.title('Parts Accuracy Evolution')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Visualizar resultados
plot_training_history(history_initial, history_fine_tune)